# Panre List Collection

In [1]:
import pandas as pd

import xml.etree.ElementTree as ET
from urllib.request import urlopen

from tqdm import trange
from tqdm import tqdm

## EDA

In [2]:
page = 1
display = 100

ID = ''
BASE = "https://www.law.go.kr/DRF/"

SEARCH_URL = f"{BASE}lawSearch.do?OC={ID}&target=prec&type=XML&display={display}&page={page}"

In [3]:
response = urlopen(SEARCH_URL).read()
xtree = ET.fromstring(response)

In [4]:
# 판례 개수
totalCnt = int(xtree.find('totalCnt').text); totalCnt

87067

In [5]:
int(totalCnt / display) + 1

871

In [6]:
page = 1
panre_list = []

for i in trange(int(totalCnt / display) + 2):
    SEARCH_URL = f"{BASE}lawSearch.do?OC={ID}&target=prec&type=XML&display={display}&page={page}"
    
    response = urlopen(SEARCH_URL).read()
    xtree = ET.fromstring(response)

    try:
        items = xtree[5:]
    except:
        break
    
    for node in items:
        판례일련번호 = node.find('판례일련번호').text
        사건명 = node.find('사건명').text
        사건번호 = node.find('사건번호').text
        선고일자 = node.find('선고일자').text
        법원명 = node.find('법원명').text
        사건종류명 = node.find('사건종류명').text
        사건종류코드 = node.find('사건종류코드').text
        판결유형 = node.find('판결유형').text
        선고 = node.find('선고').text
        판례상세링크 = node.find('판례상세링크').text

        panre_list.append({'판례일련번호': 판례일련번호,
                    '사건명': 사건명,
                    '사건번호': 사건번호,
                    '선고일자': 선고일자,
                    '법원명': 법원명,
                    '사건종류명': 사건종류명,
                    '사건종류코드': 사건종류코드,
                    '판결유형': 판결유형,
                    '선고': 선고,
                    '판례상세링크': 판례상세링크})
    
    page += 1

  0%|          | 0/872 [00:00<?, ?it/s]

100%|██████████| 872/872 [02:46<00:00,  5.24it/s]


In [7]:
len(panre_list)

87067

In [8]:
panre_df = pd.DataFrame(panre_list)
panre_df.to_csv('panre.csv', index=False)

In [20]:
panre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87200 entries, 0 to 87199
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  87200 non-null  object
 1   사건명     87200 non-null  object
 2   사건번호    87200 non-null  object
 3   선고일자    87200 non-null  object
 4   법원명     87200 non-null  object
 5   사건종류명   87200 non-null  object
 6   사건종류코드  87200 non-null  object
 7   판결유형    87200 non-null  object
 8   선고      87200 non-null  object
 9   판례상세링크  87200 non-null  object
dtypes: object(10)
memory usage: 6.7+ MB


# Panre Crawling

In [39]:
panre_df = pd.read_csv('panre.csv')

In [40]:
ID = ''
SERVICE_BASE = "https://www.law.go.kr"

SERVICE_URL = SERVICE_BASE + panre_df.loc[0]['판례상세링크'].replace('HTML', 'XML')

In [47]:
response = urlopen(SERVICE_URL).read()
xtree = ET.fromstring(response)

In [48]:
list(xtree)

[<Element '판례정보일련번호' at 0x16b3dcf40>,
 <Element '사건명' at 0x16b3f8c70>,
 <Element '사건번호' at 0x16b3f9580>,
 <Element '선고일자' at 0x16b3fa750>,
 <Element '선고' at 0x16b3fb7e0>,
 <Element '법원명' at 0x16b3fb6f0>,
 <Element '법원종류코드' at 0x16b3f9a80>,
 <Element '사건종류명' at 0x16b3f8540>,
 <Element '사건종류코드' at 0x16b3f8a40>,
 <Element '판결유형' at 0x16b3f8fe0>,
 <Element '판시사항' at 0x16b3fa890>,
 <Element '판결요지' at 0x16b3f84f0>,
 <Element '참조조문' at 0x16b3fb880>,
 <Element '참조판례' at 0x16b3f8860>,
 <Element '판례내용' at 0x16a891670>]

In [35]:
xtree[14].text

'【원고, 피상고인 겸 상고인】 주식회사 서영엔지니어링<br/>【원고(선정당사자), 피상고인 겸 상고인】 원고(선정당사자) 2 (소송대리인 법무법인 시공 담당변호사 최승진 외 2인)<br/>【피고, 상고인】 이지스자산운용 주식회사 외 1인 (소송대리인 변호사 박종욱 외 5인)<br/>【피고, 피상고인】 주식회사 에스원 (소송대리인 법무법인(유한) 광장 담당변호사 여철기 외 3인)<br/>【피고3의 보조참가인】 주식회사 대교산업 (소송대리인 법무법인 도시와사람 담당변호사 계민혜 외 4인)<br/>【원심판결】 서울고법 2019. 1. 10. 선고 2018나2014531 판결<br/>【주    문】<br/>  상고를 모두 기각한다.  상고비용 중 원고들(선정당사자 포함)과 피고 이지스자산운용 주식회사, 주식회사 국민은행 사이에 생긴 부분은 위 피고들이, 원고들(선정당사자 포함)과 피고 주식회사 에스원 사이에 생긴 부분은 보조참가로 인한 부분을 포함하여 원고들(선정당사자 포함)이 각 부담한다.<br/><br/>【이    유】  상고이유(상고이유서 제출기간이 지난 후에 제출된 상고이유보충서의 기재는 상고이유를 보충하는 범위 내에서)를 판단한다.<br/>  1. 사안의 개요<br/>  원심판결 이유와 기록에 의하면 다음 사실을 알 수 있다. <br/>  가. 피고 이지스자산운용 주식회사(이하 ‘피고 이지스자산운용’이라고 한다)는 2013. 4. 30. 「자본시장과 금융투자업에 관한 법률」에 따라 투자신탁 형식의 사모부동산집합투자기구인 이지스KORIF사모부동산투자신탁22호(이하 ‘이 사건 펀드’라고 한다)를 설정하고 신탁업자인 피고 주식회사 국민은행(이하 ‘피고 국민은행’이라고 한다)과 신탁계약을 체결하였다.<br/>  나. 피고 국민은행은 2013. 7. 4. 피고 이지스자산운용의 운용지시에 따라 이 사건 펀드의 투자신탁재산으로 성남시 분당구 (주소 생략)에 있는 ○○빌딩(이하 ‘이 사건 건물’이라고 한다)에 관한 소유권을 취득하고, 2013. 8. 22. 원고 주식회사

In [ ]:
panre_text_list = []

for i in tqdm(range(len(panre_df))):
    SERVICE_URL = SERVICE_BASE + panre_df.loc[i]['판례상세링크'].replace('HTML', 'XML')
    
    response = urlopen(SERVICE_URL).read()
    xtree = ET.fromstring(response)
    
    판례정보일련번호 = xtree.find('판례정보일련번호').text
    사건명 = xtree.find('사건명').text
    사건번호 = xtree.find('사건번호').text
    선고일자 = xtree.find('선고일자').text
    선고 = xtree.find('선고').text
    법원명 = xtree.find('법원명').text
    법원종류코드 = xtree.find('법원종류코드').text
    사건종류명 = xtree.find('사건종류명').text
    사건종류코드 = xtree.find('사건종류코드').text
    판결유형 = xtree.find('판결유형').text
    판시사항 = xtree.find('판시사항').text
    판결요지 = xtree.find('판결요지').text
    참조조문 = xtree.find('참조조문').text
    참조판례 = xtree.find('참조판례').text
    판례내용 = xtree.find('판례내용').text

    panre_text_list.append({'판례정보일련번호': 판례정보일련번호,
                '사건명': 사건명,
                '사건번호': 사건번호,
                '선고일자': 선고일자,
                '선고': 선고,
                '법원명': 법원명,
                '법원종류코드': 법원종류코드,
                '사건종류명': 사건종류명,
                '사건종류코드': 사건종류코드,
                '판결유형': 판결유형,
                '판시사항': 판시사항,
                '판결요지': 판결요지,
                '참조조문': 참조조문,
                '참조판례': 참조판례,
                '판례내용': 판례내용
        })

In [57]:
panre_text_list_df = pd.DataFrame(panre_text_list)
panre_text_list_df.to_csv('panre_text.csv', index=False)

----